In [1]:
import glob
import numpy as np
import os.path as path
from scipy import misc
import cv2

In [ ]:
images = glob.glob('trafficlight_images/*.jpg')

for image in images:
    img = cv2.imread(image)
    resizeimg = cv2.resize(img, (100, 200), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(image.split('.')[0] + '_resize.jpg', resizeimg)

In [2]:
# IMAGE_PATH should be the path to the planesnet folder
IMAGE_PATH = 'TrafficLight_Resize'
file_paths = glob.glob(IMAGE_PATH + '/*resize.jpg')

# Load the images
images = [misc.imread(path) for path in file_paths]

print('Num of pictures to train: ', len(images), 'Resolution: ', images[0].shape)

/Users/gaspards/miniconda3/envs/py27_TF1.3/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


('Num of pictures to train: ', 300, 'Resolution: ', (200, 100, 3))


In [3]:
images = np.asarray(images)

print(images.shape)

# Get image size
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

(300, 200, 100, 3)
[200 100   3]


In [4]:
# Scale
images = images / 255

# Read the labels from the filenames
n_images = images.shape[0]
labels = np.zeros(n_images)
for i in range(n_images):
    #filename = path.basename(file_paths[i])[0]
    filename = file_paths[i].split('/')[-1]
    labels[i] = int(filename[0])

In [5]:
# Split into test and training sets
TRAIN_TEST_SPLIT = 0.8

# Split at the given index
split_index = int(TRAIN_TEST_SPLIT * n_images)
shuffled_indices = np.random.permutation(n_images)
train_indices = shuffled_indices[0:split_index]
test_indices = shuffled_indices[split_index:]

In [6]:
# Split the images and the labels
x_train = images[train_indices, :, :]
y_train = labels[train_indices]
x_test = images[test_indices, :, :]
y_test = labels[test_indices]

In [7]:
## MODEL CREATION ##
# Imports
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime

Using TensorFlow backend.


In [8]:
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=image_size),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(3),
    Activation('softmax')
]

In [13]:
# create complete model
model = Sequential(feature_layers + classification_layers)

## MODEL TRAINING ##
# Training Hyperparamters
EPOCHS = 10
BATCH_SIZE = 128

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)


model.compile(loss='categorical_crossentropy',
            optimizer='adadelta',
            metrics=['accuracy'])

model.summary()

# Train the model
model.fit(x_train, y_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1)

model.save('model.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 98, 32)       896       
_________________________________________________________________
activation_1 (Activation)    (None, 198, 98, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 96, 32)       9248      
_________________________________________________________________
activation_2 (Activation)    (None, 196, 96, 32)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 98, 48, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 150528)            0         
__________

In [14]:
## MODEL EVALUATION ##
# Make a prediction on the test set
test_predictions = model.predict(x_test)
test_predictions = np.round(test_predictions)
y_one_hot_test = label_binarizer.fit_transform(y_test)

In [15]:
# Report the accuracy
accuracy = accuracy_score(y_one_hot_test, test_predictions)
print("Accuracy: " + str(accuracy))

Accuracy: 1.0


In [27]:
# Try inference couple samples
from keras.models import load_model

model = load_model('model.h5')
model.summary()

test1_img = cv2.imread('trafficlight_images/1_image10.jpg')
test1_img = cv2.resize(test1_img, (100, 200), interpolation=cv2.INTER_CUBIC)
test1_img = np.asarray(test1_img) / 255

test2_img = cv2.imread('trafficlight_images/0_image104.jpg')
test2_img = cv2.resize(test2_img, (100, 200), interpolation=cv2.INTER_CUBIC)
test2_img = np.asarray(test2_img) / 255

test3_img = cv2.imread('trafficlight_images/2_image44.jpg')
test3_img = cv2.resize(test3_img, (100, 200), interpolation=cv2.INTER_CUBIC)
test3_img = np.asarray(test3_img) / 255

test = np.array([test1_img, test2_img, test3_img])

test_pred = model.predict(test)
test_pred = np.round(test_pred)

#from sklearn.preprocessing import LabelBinarizer

result = label_binarizer.inverse_transform(test_pred)
print(result)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 98, 32)       896       
_________________________________________________________________
activation_1 (Activation)    (None, 198, 98, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 196, 96, 32)       9248      
_________________________________________________________________
activation_2 (Activation)    (None, 196, 96, 32)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 48, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 98, 48, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 150528)            0         
__________